<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [2]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [ ]:
# Kaggle Project: Dstl Satellite Imagery Feature Detection
# Name: Jianlei(John) Sun
# Beginning Date: March 7, 2017

# Library

In [1]:
__author__ = "JohnSun"

import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os
import random
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras import backend as K
from sklearn.metrics import jaccard_similarity_score
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict
import simplejson
%matplotlib inline

from joblib import Parallel, delayed

Using Theano backend.


# Input

In [5]:
N_Cls = 10
DF = pd.read_csv('./train_wkt_v4.csv')
GS = pd.read_csv('./grid_sizes.csv', names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
SB = pd.read_csv('./sample_submission.csv')
ISZ = 160
smooth = 1e-12

# Function

In [6]:
def _convert_coordinates_to_raster(coords, img_size, xymax):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    Xmax, Ymax = xymax
    H, W = img_size
    W1 = 1.0 * W * W / (W + 1)
    H1 = 1.0 * H * H / (H + 1)
    xf = W1 / Xmax
    yf = H1 / Ymax
    coords[:, 1] *= yf
    coords[:, 0] *= xf
    coords_int = np.round(coords).astype(np.int32)
    return coords_int


def _get_xmax_ymin(grid_sizes_panda, imageId):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    xmax, ymin = grid_sizes_panda[grid_sizes_panda.ImageId == imageId].iloc[0, 1:].astype(float)
    return (xmax, ymin)


def _get_polygon_list(wkt_list_pandas, imageId, cType):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    df_image = wkt_list_pandas[wkt_list_pandas.ImageId == imageId]
    multipoly_def = df_image[df_image.ClassType == cType].MultipolygonWKT
    polygonList = None
    if len(multipoly_def) > 0:
        assert len(multipoly_def) == 1
        polygonList = wkt_loads(multipoly_def.values[0])
    return polygonList


def _get_and_convert_contours(polygonList, raster_img_size, xymax):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    perim_list = []
    interior_list = []
    if polygonList is None:
        return None
    for k in range(len(polygonList)):
        poly = polygonList[k]
        perim = np.array(list(poly.exterior.coords))
        perim_c = _convert_coordinates_to_raster(perim, raster_img_size, xymax)
        perim_list.append(perim_c)
        for pi in poly.interiors:
            interior = np.array(list(pi.coords))
            interior_c = _convert_coordinates_to_raster(interior, raster_img_size, xymax)
            interior_list.append(interior_c)
    return perim_list, interior_list


def _plot_mask_from_contours(raster_img_size, contours, class_value=1):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    img_mask = np.zeros(raster_img_size, np.uint8)
    if contours is None:
        return img_mask
    perim_list, interior_list = contours
    cv2.fillPoly(img_mask, perim_list, class_value)
    cv2.fillPoly(img_mask, interior_list, 0)
    return img_mask


def generate_mask_for_image_and_class(raster_size, imageId, class_type, grid_sizes_panda=GS, wkt_list_pandas=DF):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    xymax = _get_xmax_ymin(grid_sizes_panda, imageId)
    polygon_list = _get_polygon_list(wkt_list_pandas, imageId, class_type)
    contours = _get_and_convert_contours(polygon_list, raster_size, xymax)
    mask = _plot_mask_from_contours(raster_size, contours, 1)
    return mask


def M(image_id):
    # __author__ = amaia
    # https://www.kaggle.com/aamaia/dstl-satellite-imagery-feature-detection/rgb-using-m-bands-example
    img = tiff.imread('./sixteen_band/{}_M.tif'.format(image_id))
    img = np.rollaxis(img, 0, 3)
    return img


def stretch_n(bands, lower_percent=0, higher_percent=100):
    out = np.zeros_like(bands, dtype=np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0  # np.min(band)
        b = 1  # np.max(band)
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)


def jaccard_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)


def stick_all_train():
    print("let's stick all imgs together")
    s = 835

    x = np.zeros((5 * s, 5 * s, 8))
    y = np.zeros((5 * s, 5 * s, N_Cls))

    ids = sorted(DF.ImageId.unique())
    print(len(ids))
    for i in range(5):
        for j in range(5):
            id = ids[5 * i + j]

            img = M(id)
            img = stretch_n(img)
            print(img.shape, id, np.amax(img), np.amin(img))
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1)[:s, :s]

    print(np.amax(y), np.amin(y))

    np.save('./uNet_f/input/x_trn_%d' % N_Cls, x)
    np.save('./uNet_f/input/y_trn_%d' % N_Cls, y)


def get_patches(img, msk, amt=10000, aug=True):
    is2 = int(1.0 * ISZ)
    xm, ym = img.shape[0] - is2, img.shape[1] - is2

    x, y = [], []

    tr = [0.4, 0.1, 0.1, 0.15, 0.3, 0.95, 0.1, 0.05, 0.001, 0.005]
    for i in range(amt):
        xc = random.randint(0, xm)
        yc = random.randint(0, ym)

        im = img[xc:xc + is2, yc:yc + is2]
        ms = msk[xc:xc + is2, yc:yc + is2]

        for j in range(N_Cls):
            sm = np.sum(ms[:, :, j])
            if 1.0 * sm / is2 ** 2 > tr[j]:
                if aug:
                    if random.uniform(0, 1) > 0.5:
                        im = im[::-1]
                        ms = ms[::-1]
                    if random.uniform(0, 1) > 0.5:
                        im = im[:, ::-1]
                        ms = ms[:, ::-1]

                x.append(im)
                y.append(ms)

    x, y = 2 * np.transpose(x, (0, 3, 1, 2)) - 1, np.transpose(y, (0, 3, 1, 2))
    print(x.shape, y.shape, np.amax(x), np.amin(x), np.amax(y), np.amin(y))
    return x, y


def make_val():
    print("let's pick some samples for validation")
    img = np.load('./uNet_f/input/x_trn_%d.npy' % N_Cls)
    msk = np.load('./uNet_f/input/y_trn_%d.npy' % N_Cls)
    x, y = get_patches(img, msk, amt=3000)

    np.save('./uNet_f/input/x_tmp_%d' % N_Cls, x)
    np.save('./uNet_f/input/y_tmp_%d' % N_Cls, y)


def get_unet():
    inputs = Input((8, ISZ, ISZ))
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(N_Cls, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model


def calc_jacc(model):
    img = np.load('./uNet_f/input/x_tmp_%d.npy' % N_Cls)
    msk = np.load('./uNet_f/input/y_tmp_%d.npy' % N_Cls)

    prd = model.predict(img, batch_size=4)
    print(prd.shape, msk.shape)
    avg, trs = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]
        t_msk = t_msk.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        t_prd = t_prd.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])

        m, b_tr = 0, 0
        for j in range(10):
            tr = j / 10.0
            pred_binary_mask = t_prd > tr

            jk = jaccard_similarity_score(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr
        print(i, m, b_tr)
        avg.append(m)
        trs.append(b_tr)

    score = sum(avg) / 10.0
    return score, trs


def mask_for_polygons(polygons, im_size):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


def mask_to_polygons(mask, epsilon=1, min_area=1.):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly

    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(
        ((mask == 1) * 255).astype(np.uint8),
        cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    # create approximate contours to have reasonable submission size
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons


def get_scalers(im_size, x_max, y_min):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    h, w = float(h), float(w)
    w_ = 1.0 * w * (w / (w + 1))
    h_ = 1.0 * h * (h / (h + 1))
    return w_ / x_max, h_ / y_min


def train_net():
    print("start train net")
    
    img = np.load('./uNet_f/input/x_trn_%d.npy' % N_Cls)
    msk = np.load('./uNet_f/input/y_trn_%d.npy' % N_Cls)
    x_trn, y_trn = get_patches(img, msk)
    del img, msk
    
    x_val, y_val = np.load('./uNet_f/input/x_tmp_%d.npy' % N_Cls), np.load('./uNet_f/input/y_tmp_%d.npy' % N_Cls)
    
    model = get_unet()
    model.load_weights('./uNet_f/weights/unet_f.hdf5')
    model_checkpoint = ModelCheckpoint('./uNet_f/weights/unet_f.hdf5', monitor='loss', save_best_only=True)
    for i in range(1):
        model.fit(x_trn, y_trn, batch_size=64, nb_epoch=10, verbose=1, shuffle=True,
                  callbacks=[model_checkpoint], validation_data=(x_val, y_val))
        del x_trn
        del y_trn
        x_trn, y_trn = get_patches(img, msk)
        score, trs = calc_jacc(model)
        print('val jk', score)
        
    return model


def predict_id(id, model, trs):
    img = M(id)
    x = stretch_n(img)

    cnv = np.zeros((960, 960, 8)).astype(np.float32)
    prd = np.zeros((N_Cls, 960, 960)).astype(np.float32)
    cnv[:img.shape[0], :img.shape[1], :] = x

    for i in range(0, 6):
        line = []
        for j in range(0, 6):
            line.append(cnv[i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ])

        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model.predict(x, batch_size=4)
        for j in range(tmp.shape[0]):
            prd[:, i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ] = tmp[j]

    trs = [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]
    for i in range(N_Cls):
        prd[i] = prd[i] > trs[i]

    return prd[:, :img.shape[0], :img.shape[1]]


def predict_test(model, trs = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]):
    print("predict test")
    for i, id in enumerate(sorted(set(SB['ImageId'].tolist()))):
        msk = predict_id(id, model, trs)
        np.save('./uNet_f/msk/10_%s' % id, msk)
        if i % 100 == 0: print(i, id)


def make_submit():
    print("make submission file")
    df = pd.read_csv('sample_submission.csv')
    print(df.head())
    for idx, row in df.iterrows():
        id = row[0]
        kls = row[1] - 1

        msk = np.load('./uNet_f/msk/10_%s.npy' % id)[kls]
        pred_polygons = mask_to_polygons(msk)
        x_max = GS.loc[GS['ImageId'] == id, 'Xmax'].as_matrix()[0]
        y_min = GS.loc[GS['ImageId'] == id, 'Ymin'].as_matrix()[0]

        x_scaler, y_scaler = get_scalers(msk.shape, x_max, y_min)

        scaled_pred_polygons = shapely.affinity.scale(pred_polygons, xfact=1.0 / x_scaler, yfact=1.0 / y_scaler,
                                                      origin=(0, 0, 0))

        df.iloc[idx, 2] = shapely.wkt.dumps(scaled_pred_polygons)
        if idx % 100 == 0: print(idx)
    print(df.head())
    df.to_csv('./uNet_f/uNet_f.csv', index=False)


def check_predict(id='6120_2_3'):
    model = get_unet()
    model.load_weights('./uNet_f/weights/unet_f.hdf5')

    msk = predict_id(id, model, [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1])
    img = M(id)

    plt.figure()
    ax1 = plt.subplot(131)
    ax1.set_title('image ID:6120_2_3')
    ax1.imshow(img[:, :, 5], cmap=plt.get_cmap('gist_ncar'))
    ax2 = plt.subplot(132)
    ax2.set_title('predict bldg pixels')
    ax2.imshow(msk[0], cmap=plt.get_cmap('gray'))
    ax3 = plt.subplot(133)
    ax3.set_title('predict bldg polygones')
    ax3.imshow(mask_for_polygons(mask_to_polygons(msk[0], epsilon=1), img.shape[:2]), cmap=plt.get_cmap('gray'))

    plt.show()

    
def calc_jacc(model):
    img = np.load('./uNet_f/input/x_tmp_%d.npy' % N_Cls)
    msk = np.load('./uNet_f/input/y_tmp_%d.npy' % N_Cls)

    prd = model.predict(img, batch_size=4)
    print (prd.shape, msk.shape)
    avg, trs = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]
        t_msk = t_msk.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        t_prd = t_prd.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])

        m, b_tr = 0, 0
        n_div = 100
        for j in range(n_div):
            tr = j / 10.0
            pred_binary_mask = t_prd > tr

            jk = jaccard_similarity_score(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr
        print (i+1, m, b_tr)
        avg.append(m)
        trs.append(b_tr)

    score = sum(avg) / 10.0
    return score, trs

# Preprocessing

In [5]:
stick_all_train()
make_val()

let's stick all imgs together
25
(837, 849, 8) 6010_1_2 1.0 0.0
(837, 849, 8) 6010_4_2 1.0 0.0
(837, 848, 8) 6010_4_4 1.0 0.0
(837, 848, 8) 6040_1_0 1.0 0.0
(837, 848, 8) 6040_1_3 1.0 0.0
(837, 848, 8) 6040_2_2 1.0 0.0
(837, 846, 8) 6040_4_4 1.0 0.0
(837, 851, 8) 6060_2_3 1.0 0.0
(838, 835, 8) 6070_2_3 1.0 0.0
(837, 848, 8) 6090_2_0 1.0 0.0
(837, 848, 8) 6100_1_3 1.0 0.0
(837, 848, 8) 6100_2_2 1.0 0.0
(837, 848, 8) 6100_2_3 1.0 0.0
(837, 849, 8) 6110_1_2 1.0 0.0
(837, 849, 8) 6110_3_1 1.0 0.0
(837, 849, 8) 6110_4_0 1.0 0.0
(837, 851, 8) 6120_2_0 1.0 0.0
(837, 851, 8) 6120_2_2 1.0 0.0
(837, 849, 8) 6140_1_2 1.0 0.0
(837, 849, 8) 6140_3_1 1.0 0.0
(837, 851, 8) 6150_2_3 1.0 0.0
(837, 848, 8) 6160_2_1 1.0 0.0
(837, 848, 8) 6170_0_4 1.0 0.0
(837, 848, 8) 6170_2_4 1.0 0.0
(837, 848, 8) 6170_4_1 1.0 0.0
1.0 0.0
let's pick some samples for validation
(970, 8, 160, 160) (970, 10, 160, 160) 1.0 -1.0 1.0 0.0


# Training

In [16]:
# train the model
model = train_net()

start train net
(3531, 8, 160, 160) (3531, 10, 160, 160) 1.0 -1.0 1.0 0.0
Train on 3531 samples, validate on 970 samples
Epoch 1/10
3531/3531 [==============================] - 2417s - loss: 0.0415 - jaccard_coef: 0.4874 - jaccard_coef_int: 0.5390 - acc: 0.9843 - val_loss: 0.0323 - val_jaccard_coef: 0.5370 - val_jaccard_coef_int: 0.5848 - val_acc: 0.9871
Epoch 2/10
3531/3531 [==============================] - 2486s - loss: 0.0309 - jaccard_coef: 0.5742 - jaccard_coef_int: 0.6289 - acc: 0.9876 - val_loss: 0.0307 - val_jaccard_coef: 0.5548 - val_jaccard_coef_int: 0.6032 - val_acc: 0.9877
Epoch 3/10
3531/3531 [==============================] - 2534s - loss: 0.0290 - jaccard_coef: 0.5903 - jaccard_coef_int: 0.6441 - acc: 0.9882 - val_loss: 0.0303 - val_jaccard_coef: 0.5613 - val_jaccard_coef_int: 0.6069 - val_acc: 0.9879
Epoch 4/10
3531/3531 [==============================] - 2509s - loss: 0.0282 - jaccard_coef: 0.5961 - jaccard_coef_int: 0.6500 - acc: 0.9885 - val_loss: 0.0305 - val_jacca


KeyboardInterrupt



In [17]:
model = get_unet()
model.load_weights('./uNet_f/weights/unet_f.hdf5')
score, trs = calc_jacc(model)

(970, 10, 160, 160) (970, 10, 160, 160)
1 0.914362537214 0.5
2 0.732293042353 0.2
3 0.972308005065 0.5
4 0.698436306481 0.4
5 0.717215486699 0.5
6 0.973675180981 0.6
7 0.990141707306 0.6
8 0.987518405695 0.6
9 0.991065369231 0.5
10 0.950199858959 0.4


# Prediction

In [19]:
trs = [0.5, 0.2, 0.5, 0.4, 0.5, 0.6, 0.6, 0.6, 0.5, 0.4]
model = get_unet()
model.load_weights('./uNet_f/weights/unet_f.hdf5')

def predict_test_par(id, model=model, trs = trs):
    print(id)
    msk = predict_id(id, model, trs)
    np.save('./uNet_f/msk/10_%s' % id, msk)
    
test_image_ids = sorted(set(SB['ImageId'].tolist()))
t = Parallel(n_jobs = 6)(delayed(predict_test_par)(i) for i in test_image_ids);

6010_0_1
6010_0_0
6010_0_2
6010_0_3
6010_0_4
6010_1_0
6010_1_1
6010_1_2
6010_1_3
6010_1_4
6010_2_0
6010_2_1
6010_2_2
6010_2_3
6010_2_4
6010_3_0
6010_3_1
6010_3_2
6010_3_3
6010_3_4
6010_4_0
6010_4_1
6010_4_3
6020_0_0
6020_0_1
6020_0_2
6020_0_3
6020_0_4
6020_1_0
6020_1_1
6020_1_2
6020_1_3
6020_1_4
6020_2_0
6020_2_1
6020_2_2
6020_2_3
6020_2_4
6020_3_0
6020_3_1
6020_3_2
6020_3_3
6020_3_4
6020_4_0
6020_4_1
6020_4_2
6020_4_3
6020_4_4
6030_0_0
6030_0_1
6030_0_2
6030_0_3
6030_0_4
6030_1_0
6030_1_1
6030_1_2
6030_1_3
6030_1_4
6030_2_0
6030_2_1
6030_2_2
6030_2_3
6030_2_4
6030_3_0
6030_3_1
6030_3_2
6030_3_3
6030_3_4
6030_4_0
6030_4_1
6030_4_2
6030_4_3
6030_4_4
6040_0_0
6040_0_1
6040_0_2
6040_0_3
6040_0_4
6040_1_1
6040_1_2
6040_1_4
6040_2_0
6040_2_1
6040_2_3
6040_2_4
6040_3_0
6040_3_1
6040_3_2
6040_3_3
6040_3_4
6040_4_0
6040_4_1
6040_4_2
6040_4_3
6040_4_4
6050_0_0
6050_0_1
6050_0_2
6050_0_3
6050_0_4
6050_1_0
6050_1_1
6050_1_2
6050_1_3
6050_1_4
6050_2_0
6050_2_1
6050_2_2
6050_2_3
6050_2_4
6050_3_0
6

# Submission

In [21]:
make_submit()

Self-intersection at or near point 324.72727272727275 268.54545454545456
Self-intersection at or near point 723.9375 61.3125


make submission file
    ImageId  ClassType                                    MultipolygonWKT
0  6120_2_4          1  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
1  6120_2_4          2  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
2  6120_2_4          3  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
3  6120_2_4          4  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
4  6120_2_4          5  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
0


Self-intersection at or near point 776.63636363636363 215.04545454545453
Self-intersection at or near point 824 115.33333333333333
Self-intersection at or near point 807.32000000000005 621.15999999999997
Self-intersection at or near point 468.55555555555554 120.88888888888889
Ring Self-intersection at or near point 704 153
Self-intersection at or near point 645.06896551724139 60.206896551724135
Ring Self-intersection at or near point 225 451
Self-intersection at or near point 695.5 635.5
Self-intersection at or near point 449.10000000000002 167.30000000000001
Self-intersection at or near point 534.66666666666663 3.1666666666666665
Self-intersection at or near point 147.69230769230768 565.61538461538464
Self-intersection at or near point 735 498
Self-intersection at or near point 839.5 635.5
Self-intersection at or near point 214.51612903225808 762.12903225806451
Self-intersection at or near point 158 639
Self-intersection at or near point 209.58620689655172 155.93103448275863
Self-inte

100


Self-intersection at or near point 846.69117647058829 291.58823529411762
Ring Self-intersection at or near point 400 154
Self-intersection at or near point 648.36363636363637 41.909090909090907
Self-intersection at or near point 769.5 53.75
Self-intersection at or near point 810.27999999999997 683.12
Self-intersection at or near point 100 254
Self-intersection at or near point 179 728
Self-intersection at or near point 594.93939393939399 346.5151515151515
Self-intersection at or near point 782.66666666666663 572
Self-intersection at or near point 437 1
Self-intersection at or near point 822 833
Self-intersection at or near point 153.40000000000001 38
Self-intersection at or near point 143.80000000000001 35
Self-intersection at or near point 714.47368421052636 89.84210526315789
Self-intersection at or near point 639.68656716417911 2.9701492537313432
Self-intersection at or near point 726.76470588235293 302.05882352941177
Self-intersection at or near point 781.75 408.75
Self-intersection

...

4100


Self-intersection at or near point 763.83333333333337 411.66666666666669
Self-intersection at or near point 763.31578947368416 186.52631578947367
Self-intersection at or near point 485.5625 813.625
Self-intersection at or near point 653.85714285714289 713.71428571428567
Self-intersection at or near point 841.25 46.75
Self-intersection at or near point 840.757281553398 664.87378640776694
Self-intersection at or near point 664.5 1.5
Self-intersection at or near point 652 352.80000000000001
Self-intersection at or near point 842 459
Self-intersection at or near point 809.36000000000001 534.88
Self-intersection at or near point 508.85714285714283 217.71428571428572
Self-intersection at or near point 54 287
Self-intersection at or near point 811.27499999999998 210.02500000000001
Self-intersection at or near point 661 481.5
Ring Self-intersection at or near point 675 258
Self-intersection at or near point 702.08108108108104 359.2162162162162
Self-intersection at or near point 792.25 409.5
Ri

4200


Self-intersection at or near point 679.79999999999995 275.60000000000002
Self-intersection at or near point 838.21052631578948 202.34210526315789
Self-intersection at or near point 158.2608695652174 22.739130434782609
Self-intersection at or near point 565.13793103448279 89.34482758620689
Self-intersection at or near point 706.5 140
Self-intersection at or near point 828.30769230769226 427.23076923076923
Self-intersection at or near point 19 677
Self-intersection at or near point 819.5 717.5
Self-intersection at or near point 299 138
Self-intersection at or near point 478.625 372.375
Self-intersection at or near point 691.5 784
Self-intersection at or near point 55 196
Self-intersection at or near point 693 155.40000000000001
Self-intersection at or near point 561.25 109
Self-intersection at or near point 710.91304347826087 522.56521739130437
Self-intersection at or near point 845.15999999999997 324.68000000000001
Self-intersection at or near point 316.89999999999998 115.09999999999999

    ImageId  ClassType                                    MultipolygonWKT
0  6120_2_4          1  MULTIPOLYGON (((0.0091258599807714 -0.00298449...
1  6120_2_4          2  MULTIPOLYGON (((0.0091366854374508 -0.00389281...
2  6120_2_4          3  MULTIPOLYGON (((0.0091366854374508 -0.00346028...
3  6120_2_4          4  MULTIPOLYGON (((0.0089634781305797 -0.00067042...
4  6120_2_4          5  MULTIPOLYGON (((0.0091691618074892 -0.00196803...


In [13]:
# update training labels

overlap_list = ['6010_1_2', '6040_4_4', '6070_2_3', '6100_2_2']
overlapData = pd.DataFrame()
for i in overlap_list:
    overlapData = pd.concat([overlapData, DF[DF['ImageId'] == i]], axis = 0)

new = pd.read_csv('./uNet_f/uNet_f.csv') 
pd_output = pd.merge(new, overlapData, how = 'left', on = ['ImageId','ClassType'])
pd_output = pd_output.fillna(-999)

## In the case empty multipolygons were predicted, use default polygons from sample submission
pd_output ['MultipolygonWKT'] = pd_output[['MultipolygonWKT_x','MultipolygonWKT_y']].apply(
    lambda x:x[0] if x[1] == -999 else x[1],axis=1)

pd_output[['ImageId','ClassType','MultipolygonWKT']].to_csv("./uNet_f/uNet_f_f.csv", index = False)